In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.keys import Keys
import pandas as pd


import time



In [3]:
options = Options()
options.add_argument('--start-maximized')
driver = webdriver.Chrome(options=options)
driver.get('https://claro.qualtrics.com/Q/MyProjectsSection')


In [4]:
# Preenche login
driver.find_element(By.ID, 'username').send_keys('93239859')
driver.find_element(By.ID, 'password').send_keys('N1nh@*25')
driver.find_element(By.ID, "signOnButton").click()

In [5]:
logado = False
while not logado:
    try:
        elemento = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, "Projetos e programas")))
        logado = True
    except:
        pin = elemento = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, "/html/body/div/div/div[1]/div/div[2]/div[3]")))
        pin = pin.text.strip()  



In [21]:
driver.get('https://claro.qualtrics.com/reporting-dashboard/web/6616d90f5c152200082e2233/pages/Page_4bccad9e-f0d7-4265-8a8c-22b831245106/view')


In [35]:
filtroEps = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[4]/main/div[2]/div[1]/div[1]/div[1]/div/div/div/ul/li[2]/span/span/span[2]/span[15]/span/div/div/div/div/div/div/div/div/div/span/button/span[2]')))
filtroEps.click()
TEL_CENTRO = 'sub0_23'
TEL_TELEMATICA = 'sub0_24'
TELEMATICA_FEIRADESATANA_DACO1 = 'sub0_25'

tel_centro = WebDriverWait(driver, 60).until(
    EC.element_to_be_clickable(
        (By.XPATH, "//a[contains(., 'TEL CENTRO')]")
    )
)
tel_centro.click()

tel_telematica = WebDriverWait(driver, 60).until(
    EC.element_to_be_clickable(
        (By.XPATH, "//a[contains(., 'TEL TELEMATICA')]")
    )
)
tel_telematica.click()

tel_telematicadac01 = WebDriverWait(driver, 60).until(
    EC.element_to_be_clickable(
        (By.XPATH, "//a[contains(., 'TELEMATICA|FEIRADESANTANA|DAC01')]")
    )
)
tel_telematicadac01.click()






In [49]:
filtroPeriodo = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[4]/main/div[2]/div[1]/div[1]/div[1]/div/div/div/ul/li[2]/span/span/span[2]/span[2]/span/div/div/span/button/span[2]')))
filtroPeriodo.click()

botao_periodo = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable(
        (By.XPATH, "//button[@aria-label='Data Resposta seleção Esta semana']")
    )
)
botao_periodo.click()

periodoUltimos7Dias = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.ID, 'sub0_3')))

periodoUltimos7Dias.click()


In [74]:
xpath_btn = '/html/body/div[1]/div[4]/main/div[2]/div[2]/div/div/div/div/div/div[4]/div[1]/div/div/div[3]/div/ng-transclude/div/dashboards.core.layout-engine/div/div/div/div/div/div[14]/div/div/div/div/div/div/div[2]/div/div/div/div[1]/h3/div/div[2]/div/div/button/div[1]'

found = False
for i in range(50):
    try:
        element = driver.find_element(By.XPATH, xpath_btn)
        driver.execute_script("arguments[0].scrollIntoView(true);", element)

        # Espera até estar clicável
        element = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, xpath_btn))
        )

        driver.execute_script("arguments[0].click();", element)
        found = True
        break

    except (NoSuchElementException, TimeoutException):
        driver.execute_script("document.scrollingElement.scrollBy(0, 600);")
        time.sleep(1)

if not found:
    raise Exception("❌ Elemento não encontrado após 50 tentativas")


In [75]:

qtd_feedback  = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[5]/main/div[2]/div[2]/div/div/div/div/div/div[4]/div[1]/div/div/div[3]/div/ng-transclude/div/dashboards.core.layout-engine/div/div/div/div/div/div[14]/div/div/div/div/div/div/div[2]/div/div/div/div[1]/h3/div/div[2]/div/div/button/div[2]")))
qtd_feedback = qtd_feedback.text
print(qtd_feedback.strip())


6


In [78]:

# Espera até que todas as respostas estejam carregadas
respostas = WebDriverWait(driver, 15).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.styles__response___\\+0-OI"))
)

for resposta in respostas:
    # Pega o aria-label da resposta
    aria_label = resposta.get_attribute("aria-label")
    
    # Pega o número da resposta
    numero = resposta.find_element(By.CSS_SELECTOR, "div.styles__ticker-content___vHHyl").text.strip()
    print(f"Número: {numero} | Label: {aria_label}")

    try:
        # Encontra o botão 'Menu' dentro desta resposta
        vizualizar_btn = resposta.find_element(By.CSS_SELECTOR, "button[aria-label='Menu']")
        
        # Scroll até o botão (evita problemas de overlay)
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", vizualizar_btn)
        
        # Espera até que o botão esteja clicável
        WebDriverWait(driver, 5).until(EC.element_to_be_clickable(vizualizar_btn))
        
        # Clica no botão usando Selenium normal (ou JS se ainda tiver problema)
        vizualizar_btn.click()

        print(f"Cliquei no menu da resposta {numero}")


                # Agora espera e clica na opção 'Ver registo completo'
        ver_registro = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, ".//span[text()='Ver registo completo']"))
        )
        ver_registro.click()
        time.sleep(10)
        # Agora espera e clica na opção 'Ver registo completo'
        fechar_registro = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "/html/body/div[90]/div[1]/div/div/div[3]/button"))
        )
        fechar_registro.click()
        print(f"Cliquei em 'Ver registo completo' da resposta {numero}")
        


    except Exception as e:
        print(f"Não consegui clicar no menu da resposta {numero}: {e}")


Número: 0 | Label: Resposta: 1. DAC. PORTO ALEGRE. há 21 horas. A atendente não foi capaz de resolver minha solicitação como se não soubesse e não é a primeira vez que necessito de um atendimento e a um problema e os atendentes não sabem resolver. 
Cliquei no menu da resposta 0
Cliquei em 'Ver registo completo' da resposta 0
Número: 10 | Label: Resposta: 2. 10. DAC. FLORIANOPOLIS. há um dia. A boa atenção prestada
Cliquei no menu da resposta 10
Cliquei em 'Ver registo completo' da resposta 10
Número: 6 | Label: Resposta: 3. 6. DAC. SERRA. há um dia. Muito defeito. Rede instável.  Teste de desligar roteador é ridículo.  Ja perdi a conta da quantidade de vezes que já fiz isto.
Cliquei no menu da resposta 6
Cliquei em 'Ver registo completo' da resposta 6
Número: 10 | Label: Resposta: 4. 10. DAC. GOIANIA. há um dia
Cliquei no menu da resposta 10
Cliquei em 'Ver registo completo' da resposta 10
Número: 6 | Label: Resposta: 5. 6. DAC. RIO DE JANEIRO. há um dia. dificuldade em falar diretamen